# A visual exploration of the size and GC content of sequenced genomes avaliable on [NCBI](https://www.ncbi.nlm.nih.gov/)

   Genome size is commonly reported in megabase pairs (megabase = 1,000,000 base pairs of DNA). The size of genomes is extremly variable across species and influenced by a large number of factors (if you're curious the human genome is ~3,000 Mb (megabase pairs), or 3 billion base pairs in size). The NCBI sequenced genome dataset includes genomes from three domains of organisms: eukaryotes (multi cellular organisms, like you, ducks and ferms), porkaryote (single cell, bacteria) and virus genomes (not a Linnean doman... [more a weird grey area](https://www.scientificamerican.com/article/are-viruses-alive-2004/). Here I plot the size of the genomes in these three domains to compare them, and also look at the GC content of the genomes across these domains. %GC content is the percentage of DNA base pairs composed of G and C (guanine and cytosine) GC-content is indicative of many protein coding genes, and the %GC content is indicative of a gene-rich genome. GC ratios within a genome are know to be extremly variable, with low %GC indicating a large amount of non-coding DNA in the genome. 

These data are derived from [NCBI's repository of sequenced genomes](https://www.ncbi.nlm.nih.gov/home/genomes/). Through the links provided in the Kaggle dataset, one can access the entire sequence to any of these genomes as they are pubically avaliable (for free! thanks science!)

## Loading the data, assessment and cleanup


In [1]:
library('tidyverse')
library('ggthemes')

In [2]:
raw_eukaryote = read_csv('../input/eukaryotes.csv')
head(raw_eukaryote)

In [3]:
# just the chromosome and scaffold level assembies, the contigs removed
eukaryote = raw_eukaryote[raw_eukaryote$Level == "Chromosome" || raw_eukaryote$Level == "Scaffold",]
head(eukaryote)

### read in the eukaryote data

In [4]:
eukaryote = eukaryote[eukaryote$"GC%" > 5,]
#Prototheca stagnorum has 71% GC content which is pretty neat
eukaryote[eukaryote$"GC%" > 70,]

In [5]:
#looking at the data there appears to be some missing data.
# GC of zero seen, looks like one with with a very low number,
# also an outlier with >70% gc, I should look at what this one is 
qplot(eukaryote$"GC%") +
    labs(title = "Histogram of Eukaryote genome %GC content", x= "%GC content", y = "Frequency") +
	theme_fivethirtyeight()

### read in the prokaryote data

In [6]:
prokaryote = read_csv('../input/prokaryotes.csv')
prokaryote = prokaryote[prokaryote$"GC%" > 5,]

In [7]:
#check to make sure things aren't valuable
low_prokaryote = prokaryote[prokaryote$"GC%" < 20,]
head(low_prokaryote)

In [8]:
#wider distribution for prokaryotes
# higher mean and median
qplot(prokaryote$"GC%") +
    labs(title = "Histogram of Prokaryote genome %GC content", x= "%GC content", y = "Frequency") +
	theme_fivethirtyeight()

### read in the virus data

In [9]:
#one 0 in the viruses... also a gc as high as 80+ investigate further
viruses = read_csv('../input/viruses.csv')
viruses = viruses[viruses$"GC%" > 5,]

In [10]:
qplot(viruses$"GC%")+
    labs(title = "Histogram of Virus genome %GC content", x= "%GC content", y = "Frequency") +
	theme_fivethirtyeight()

## %GC content - comparison of means and medians

As a first step we can assess the means and medians for the different organism types.

In [11]:
mean(eukaryote$"GC%")

In [12]:
mean(prokaryote$"GC%")

In [13]:
#mean of this GC in between the two others
mean(viruses$"GC%")

In [14]:
median(eukaryote$"GC%")

In [15]:
median(prokaryote$"GC%")

In [16]:
median(viruses$"GC%")

## Comparing trends in the genomes of Viruses, Prokaryotes and Eukaryotes

Here I put all three organism types into a single dataframe for comparing the GC content and genome size (mb) across the domains.

In [54]:
all_genomes = data.frame(domain= "virus", 
							GC = viruses$"GC%", 
							size_mb = viruses$"Size(Mb)" )

all_genomes = rbind(all_genomes, data.frame(domain="prokaryote", 
												GC = prokaryote$"GC%", 
												size_mb = prokaryote$"Size(Mb)"))
all_genomes = rbind(all_genomes, data.frame(domain="eukaryote", 
												GC = eukaryote$"GC%", 
												size_mb = eukaryote$"Size(Mb)"))

First a simple boxplot of the gc content across the categories

In [18]:
boxplot(GC ~ domain , data = all_genomes,
            main = "%GC content of sequenced Virus, Prokaryote and Eukaryote genomes")

I find that by overlaying a scatter plot on top of the boxplot, we can visualize the distribution of the data more effectively. Below I accomplish this using ggplot to produce a boxplt via `geom_boxplot` and then I overlay translucent, jittered (moved to not overlap to heavily) dots via `geom_point`. The rest of the lines are visual blandishment.

In [19]:
#plot of the gc content by domain
ggplot(all_genomes, aes(x=domain, y=GC, fill=domain, color=domain))+
	geom_boxplot(outlier.shape = NA) +
	geom_point(shape = 21, alpha = 0.1, position=position_jitterdodge())+
	scale_fill_manual(values = rep(NA,3)) +
	theme_fivethirtyeight() +
	scale_colour_economist() +
    labs(title = "%GC content of sequenced Virus, Prokaryote and Eukaryote genomes", 
            y = "%GC content",
            x = "domain")

Here I run a simple analysis of variance to compare the mean %GC content across the domains.

H0 = The mean %GC content is the same across the domains
Ha = The mean %GC content differs across the domains

In [20]:
gc_anova = aov(GC ~ domain , data=all_genomes)
summary(gc_anova)

Therefore we can reject the null hypothesis.

Using a Tukey's honest significant difference test we can then assess which categories have significant differences from one another.

In [21]:
TukeyHSD(gc_anova)

From those results we can conclude that there is no significant difference between the virus and eukaryote %GC content, while the prokaryote %GC content significantly differs from the other two groups.

Below I repeat the ANOVA process with an analysis of genome size across the three genome categorie.

In [22]:
#differences in size? should be obvious
mb_anova = aov(size_mb ~ domain , data=all_genomes)
summary(mb_anova)

Again we can assess where the significant differences lie

In [23]:
TukeyHSD(mb_anova)

The virus and prokaryote categories are on average not significantly different in terms of genome size, while the mean eukaryote genome size is significantly different from the other two groups.

In [24]:
ggplot(all_genomes, aes(x=size_mb, y=GC, shape=domain, colour=domain)) +
        geom_point() +
        scale_colour_brewer(palette="Set1") +
        coord_trans(x = "log10") +
        labs(title = "GC content vs genome size (mb) of genome sequences stored on NCBI", 
        		y = "%GC content", 
        		x = "Genome size (mb) - Note this axis is log scaled") +
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

Here I have [a large chunk of code](https://gist.github.com/dgrtwo/eb7750e74997891d7c20#file-geom_flat_violin-r) that adds a new plot type into the libary of avaliable ggplot options. It lets us create a half violin plot.

In [25]:
#######
## load source code
#locally:
# source("geom_flat_violin.R")

## sourced from github "dgrtwo/geom_flat_violin.R
## https://gist.github.com/dgrtwo/eb7750e74997891d7c20#file-geom_flat_violin-r

# I have pasted in the raw function below for the use in this kernel
#######


"%||%" <- function(a, b) {
  if (!is.null(a)) a else b
}

geom_flat_violin <- function(mapping = NULL, data = NULL, stat = "ydensity",
                        position = "dodge", trim = TRUE, scale = "area",
                        show.legend = NA, inherit.aes = TRUE, ...) {
  layer(
    data = data,
    mapping = mapping,
    stat = stat,
    geom = GeomFlatViolin,
    position = position,
    show.legend = show.legend,
    inherit.aes = inherit.aes,
    params = list(
      trim = trim,
      scale = scale,
      ...
    )
  )
}

#' @rdname ggplot2-ggproto
#' @format NULL
#' @usage NULL
#' @export
GeomFlatViolin <-
  ggproto("GeomFlatViolin", Geom,
          setup_data = function(data, params) {
            data$width <- data$width %||%
              params$width %||% (resolution(data$x, FALSE) * 0.9)
            
            # ymin, ymax, xmin, and xmax define the bounding rectangle for each group
            data %>%
              group_by(group) %>%
              mutate(ymin = min(y),
                     ymax = max(y),
                     xmin = x,
                     xmax = x + width / 2)
          },
          
          draw_group = function(data, panel_scales, coord) {
            # Find the points for the line to go all the way around
            data <- transform(data, xminv = x,
                              xmaxv = x - violinwidth * (xmax - x)) # change x - violinwidth to + for violin to right
            
            # Make sure it's sorted properly to draw the outline
            newdata <- rbind(plyr::arrange(transform(data, x = xminv), y),
                             plyr::arrange(transform(data, x = xmaxv), -y))
            
            # Close the polygon: set first and last point the same
            # Needed for coord_polar and such
            newdata <- rbind(newdata, newdata[1,])
            
            ggplot2:::ggname("geom_flat_violin", GeomPolygon$draw_panel(newdata, panel_scales, coord))
          },
          
          draw_key = draw_key_polygon,
          
          default_aes = aes(weight = 1, colour = "grey20", fill = "white", size = 0.5,
                            alpha = NA, linetype = "solid"),
          
          required_aes = c("x", "y")
)

## Violin and scatter plot of the %GC content 
Below is a new plot style I have been experimenting with, utilizing the code we just ran above.  it is essentially half of a normal [violin plot](http://ggplot2.tidyverse.org/reference/geom_violin.html) and half a jittered scatter plot. I find that this does a good job of showing the distribution of the data via two parallel methods! Below I use it to plot the distribution of %GC content across the three groups of organisms. The violin shows the general distribution, while the scatter plot shows where the individual points fall.


In [26]:


ggplot(all_genomes, aes(x = domain, y = GC, fill=domain)) +
	geom_flat_violin( colour="white") +
	geom_point(aes(x = as.numeric(domain) + .12, colour=domain) , 
					size = 0.01, 
					pch=21,
                    alpha = 0.1,
					position = position_jitterdodge(jitter.width = .55, jitter.height = 0.01, dodge.width = 0.75)) +
	theme_fivethirtyeight() +
    labs(title = "GC content of genome sequences stored on NCBI", 
        	y = "%GC content", 
        	x = "Genome size (mb)")

## Further Analysis and Understanding of GC Content

Determining how focusing on GC content above 30% differs from the analysis from above. 

In [34]:
#Read in eukaryotes data above 30
eukaryotes_30 = raw_eukaryote[raw_eukaryote$"GC%" > 30,]
head(eukaryotes_30)

In [35]:
#Read in prokaryotes data above 30 
prokaryotes_30 = prokaryote[prokaryote$"GC%" > 30,]
head(prokaryotes_30)

In [36]:
#Read in viruses above 30 
viruses_30 = viruses[viruses$"GC%" > 30,]
head(viruses_30)

## Comparing mean and median for GC content above 30

In [38]:
#Mean for eukaryotes 
mean(eukaryotes_30$"GC%")

In [39]:
#Mean for prokaryotes
mean(prokaryotes_30$"GC%")

In [42]:
#Mean for viruses
mean(viruses_30$"GC%")

Findings:
The mean for the eukaryotes is higher than viruses but lower than prokaryotes.  

In [43]:
#Median for eukaryotes
median(eukaryotes_30$"GC%")

In [44]:
#Median for prokaryotes
median(prokaryotes_30$"GC%")

In [45]:
#Median for viruses
median(viruses_30$"GC%")

Findings: The median for eukaryotes is higher than viruses but lower than prokaryotes. Compared to the previous calculated mean and median, prokaryotes have the highest for both. 

## Max for GC content above 30

In [46]:
max(eukaryotes_30$"GC%")

In [47]:
max(prokaryotes_30$"GC%")

In [48]:
max(viruses_30$"GC%")

Interesting that the viruses actually contain the highest GC content out of all the classifications. 

In [60]:
all_genomes = data.frame(domain= "virus", 
							GC = viruses_30$"GC%", 
							size_mb = viruses_30$"Size(Mb)" )

all_genomes = rbind(all_genomes, data.frame(domain="prokaryote", 
												GC = prokaryotes_30$"GC%", 
												size_mb = prokaryotes_30$"Size(Mb)"))
all_genomes = rbind(all_genomes, data.frame(domain="eukaryote", 
												GC = eukaryotes_30$"GC%", 
												size_mb = eukaryotes_30$"Size(Mb)"))

In [61]:
boxplot(GC ~ domain , data = all_genomes,
            main = "%GC content above 30 of sequenced Virus, Prokaryote and Eukaryote genomes")

In [62]:
ggplot(all_genomes, aes(x=size_mb, y=GC, shape=domain, colour=domain)) +
        geom_point() +
        scale_colour_brewer(palette="Set1") +
        coord_trans(x = "log10") +
        labs(title = "GC content vs genome size (mb) of genome sequences stored on NCBI", 
        		y = "%GC content", 
        		x = "Genome size (mb) - Note this axis is log scaled") +
        theme(axis.text.x = element_text(angle = 90, hjust = 1))